In [6]:
!pip install transformers==3.0.2

     |████████████████████████████████| 769 kB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 59.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 50.1 MB/s eta 0:00:01
     |████████████████████████████████| 862 kB 47.4 MB/s eta 0:00:01
     |████████████████████████████████| 722 kB 50.7 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.44-py3-none-any.whl size=886084 sha256=f46a900b2c022e4865c41f8d75729450e40a2e33680e8e4f652ebb375691445b
  Stored in directory: /root/.cache/pip/wheels/d4/6d/ad/81106f259084ee9e99156f754f8a4957e4c2cb9c1ccf866f8a
Successfully built sacremoses
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import transformers

from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from transformers import BertTokenizer


In [8]:
bert_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_name,
                                          add_special_tokens=True,
                                          do_lower_case=True,
                                          max_length=256,
                                          pad_to_max_length=True)

In [9]:
def bert_encoder(input_text):
    # txt = input_text.numpy().decode('utf-8')
    txt = input_text
    encoded = tokenizer.encode_plus(txt, add_special_tokens=True, 
                                    max_length=256, 
                                    pad_to_max_length=True, 
                                    return_attention_mask=True, 
                                    return_token_type_ids=True,
                                    truncation=True)
    return encoded['input_ids'], encoded['token_type_ids'], \
           encoded['attention_mask']

def build_model(transformer, max_len=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='relu', kernel_initializer='he_normal')(cls_token)
    model = Model(inputs=input_word_ids, outputs=out)
    for layer in model.layers[:-1]:
      layer.trainable = False
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    model.compile(optimizer = optimizer, 
                  loss=tf.keras.metrics.mean_squared_error,
                  metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

MAX_LEN = 256
def create_model():
  transformer_layer = (
      transformers.TFAutoModelWithLMHead.from_pretrained(bert_name)
  )
  model = build_model(transformer_layer, max_len=MAX_LEN)
  return model


In [25]:
checkpoint_path = "/tf/saved_model/checkpoints/cp.ckpt"
model_reload = create_model()
model_reload.load_weights(checkpoint_path)
model_reload.summary()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForMaskedLM: ['nsp___cls']
- This IS expected if you are initializing TFBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 256)]             0         
_________________________________________________________________
tf_bert_for_masked_lm (TFBer ((None, 256, 30522),)     110104890 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 30522)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 30523     
Total params: 110,135,413
Trainable params: 30,523
Non-trainable params: 110,104,890
_________________________________________________________________


In [7]:
tf.saved_model.save(model_reload, export_dir='saved_model/michael/BERT')

INFO:tensorflow:Assets written to: saved_model/michael/BERT/assets


INFO:tensorflow:Assets written to: saved_model/michael/BERT/assets


In [26]:
transcription_txt_file_path = "/tf/data/" + "transcription_original_id_txt_mmse.csv"
filtered_merged_data = pd.read_csv(transcription_txt_file_path)
filtered_merged_data.head()

,Unnamed: 0,ID,Age,Gender,TEXT,MMSE,Group
0,0,S001,74,male,well there's a mother standing there washing ...,30,cc
1,1,S002,62,female,somebody's getting cookies out_of the cookie ...,30,cc
2,2,S003,69,female,okay . there's a little boy and he's standing...,29,cc
3,3,S004,71,female,are you ready ? well the sink is overflowing ...,30,cc
4,4,S005,74,female,okay . the mother's washing the dishes and th...,30,cc


In [27]:
filtered_merged_data.TEXT[0]

" well there's a mother standing there washing the dishes and the sink is overflowing . and the window's open . and outside the window there's a curved walk with a garden . and you can see another building there . looks like a garage or something with curtains and the grass in the garden . and there are two cups and a saucer on the sink . and she's getting her feet wet from the overflow of the water from the sink . she seems to be oblivious to the fact that the sink is overflowing . she's also oblivious to the fact that her kids are stealing cookies out of the cookie jar . and the kid on the stool is gonna fall off the stool . he's standing up there in the cupboard taking cookies out of the jar , handing them to a girl about the same age . the kids are somewhere around seven or eight years old or nine . and the mother is gonna get shocked when he tumbles and the cookie jar comes down . and I think that's about all ."

In [69]:
bert_train = [bert_encoder(r) for r in filtered_merged_data["TEXT"]]
bert_lbl = filtered_merged_data["MMSE"]
bert_train = np.array(bert_train)
sc_reviews, sc_segments, sc_masks = np.split(bert_train, 3, axis=1)


In [70]:
predict = model_reload.predict(sc_reviews.squeeze(), batch_size=64)
filtered_merged_data['predict'] = [x[0] if x[0] <= 30 else 30 for x in predict]
filtered_merged_data.head()

,Unnamed: 0,ID,Age,Gender,TEXT,MMSE,Group,predict
0,0,S001,74,male,well there's a mother standing there washing ...,30,cc,26.000887
1,1,S002,62,female,somebody's getting cookies out_of the cookie ...,30,cc,28.357553
2,2,S003,69,female,okay . there's a little boy and he's standing...,29,cc,25.372938
3,3,S004,71,female,are you ready ? well the sink is overflowing ...,30,cc,23.448494
4,4,S005,74,female,okay . the mother's washing the dishes and th...,30,cc,26.735819


In [79]:
sc_reviews.squeeze()

array([[  101,  2092,  2045, ...,     0,     0,     0],
       [  101,  8307,  1005, ...,     0,     0,     0],
       [  101,  3100,  1012, ...,     0,     0,     0],
       ...,
       [  101,  2821,  2017, ...,     0,     0,     0],
       [  101,  2017,  2215, ...,     0,     0,     0],
       [  101,  1049, 14227, ...,     0,     0,     0]])

In [99]:
np.array(sc_reviews.squeeze()[0]).reshape(1, 256).shape

(256,)

In [101]:
model_reload.predict(np.array(sc_reviews.squeeze()[0]).reshape(1, 256))

array([[26.000887]], dtype=float32)

In [135]:
filtered_merged_data[filtered_merged_data.ID == 'S041']

,Unnamed: 0,ID,Age,Gender,TEXT,MMSE,Group,predict
32,32,S041,57,female,water's pouring out_of the sink . and the wom...,30,cc,26.420732


In [134]:
!aws s3 sync /tf/saved_model/michael/BERT s3://w210-audio-files-bucket/bert

upload: saved_model/michael/BERT/variables/variables.index to s3://w210-audio-files-bucket/bert/variables/variables.index
upload: saved_model/michael/BERT/saved_model.pb to s3://w210-audio-files-bucket/bert/saved_model.pb
upload: saved_model/michael/BERT/variables/variables.data-00000-of-00001 to s3://w210-audio-files-bucket/bert/variables/variables.data-00000-of-00001


# Test serving

In [17]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 21.7 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [26]:
!pip install pydub

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [37]:
import speech_recognition as sr
import os
import shutil

In [28]:
file_path = "/home/ubuntu/model/data/S041.wav"
            
audio_path = "/tf/data/" + os.path.basename(file_path)

In [44]:
# initialize the speech recognizer
r = sr.Recognizer()

# a function that splits the audio file into chunks 
# and applies speech recognition 
def silence_based_conversion(path, wav_file): 
    text_df = pd.DataFrame()
  
    # open the audio file stored in 
    # the local system as a wav file. 
    song = AudioSegment.from_wav(path + wav_file) 
    print(song)
  
    # open a file where we will concatenate   
    # and store the recognized text 
    text_file = wav_file.partition('.')[0] + ".txt"
    text_file_dir = path + "output_text/" 
    if not os.path.exists(text_file_dir):
        os.makedirs(text_file_dir)    
    text_file_path = text_file_dir + text_file
    fh = open(text_file_path, "w+") 
          

    dBFS = song.dBFS
    print('dBFS: ' + str(dBFS))
    # chunks = split_on_silence(song, 
    #     min_silence_len = 500,
    #     silence_thresh = dBFS-16,
    #     keep_silence = 250 
    # )

    # split track where silence is 0.5 seconds  
    # or more and get chunks 
    chunks = split_on_silence(song, 
        # must be silent for at least 0.5 seconds 
        # or 500 ms. adjust this value based on user 
        # requirement. if the speaker stays silent for  
        # longer, increase this value. else, decrease it. 
        min_silence_len = 250, 
  
        # consider it silent if quieter than -16 dBFS 
        # adjust this per requirement 
        silence_thresh = dBFS - 16
        # keep_silence = 250
    ) 
    # setting minimum length of each chunk to 25 seconds
    target_length = 20 * 1000 
    output_chunks = [chunks[0]]
    for chunk in chunks[1:]:
      print('Length of chunk: ' + str(len(output_chunks[-1])) )
      if len(output_chunks[-1]) < target_length:
        output_chunks[-1] += chunk
      else:
        # if the last output chunk is longer than the target length,
        # we can start a new one
        output_chunks.append(chunk)    
    # print(chunks)
  
    # create a directory to store the audio chunks. 
    try: 
        os.mkdir(path + 'bert_audio_chunks') 
    except(FileExistsError): 
        pass
  
    # move into the directory to 
    # store the audio files. 
    os.chdir(path +'bert_audio_chunks') 
  
    i = 0
    transcript = ''
    # process each chunk 
    for chunk in output_chunks: 
              
        # Create 0.5 seconds silence chunk 
        chunk_silent = AudioSegment.silent(duration = 10) 
  
        # add 0.5 sec silence to beginning and  
        # end of audio chunk. This is done so that 
        # it doesn't seem abruptly sliced. 
        audio_chunk = chunk_silent + chunk + chunk_silent 
  
        # export audio chunk and save it in  
        # the current directory. 
        text_file_id = text_file.partition('.')[0]
        chunk_file_name = text_file_id + "_" + "chunk" + str(i) + ".wav"
        print("saving " + chunk_file_name) 
        # specify the bitrate to be 192 k 
        audio_chunk.export("./" + chunk_file_name, bitrate ='192k', format ="wav") 
  
        # the name of the newly created chunk 
        filename = chunk_file_name
  
        print("Processing chunk file: " + filename) 
  
        # get the name of the newly created chunk 
        # in the AUDIO_FILE variable for later use. 
        file = filename 
  
        # create a speech recognition object 
        r = sr.Recognizer() 
  
        # recognize the chunk 
        with sr.AudioFile(file) as source: 
            # remove this if it is not working 
            # correctly. 
            #r.adjust_for_ambient_noise(source) 
            audio_listened = r.record(source) 
  
        try: 
            # try converting it to text 
            rec = r.recognize_google(audio_listened) 
            # write the output to the file. 
#             fh.write(rec+". ") 
#             text_df = text_df.append({'ID': text_file_id, 'Text': rec}, ignore_index = True)
            transcript = transcript + rec + ' '
        # catch any errors. 
        except sr.UnknownValueError: 
            print("Could not understand audio") 
  
        except sr.RequestError as e: 
            print("Could not request results. check your internet connection") 
  
        i += 1
    shutil.rmtree(path + 'bert_audio_chunks')     
    return transcript

# silence_based_conversion(audio_path_cc, 'S001.wav')        
# text_df = silence_based_conversion(audio_path_cd, 'S079.wav')        
# print(text_df)
# silence_based_conversion(audio_path_cc + 'spkr0.wav')        

In [48]:
transcript = silence_based_conversion("/tf/data/" , os.path.basename(file_path))      

dBFS: -30.995559274792225
Length of chunk: 2185
Length of chunk: 4442
Length of chunk: 10851
Length of chunk: 13546
Length of chunk: 15626
Length of chunk: 19261
Length of chunk: 20252
Length of chunk: 1474
Length of chunk: 3874
Length of chunk: 5254
Length of chunk: 7643
Length of chunk: 8832
Length of chunk: 12535
Length of chunk: 13921
Length of chunk: 14996
Length of chunk: 19556
Length of chunk: 22148
Length of chunk: 1132
Length of chunk: 2829
Length of chunk: 4079
Length of chunk: 8412
Length of chunk: 9717
saving S041_chunk0.wav
Processing chunk file: S041_chunk0.wav
saving S041_chunk1.wav
Processing chunk file: S041_chunk1.wav
saving S041_chunk2.wav
Processing chunk file: S041_chunk2.wav


In [49]:
transcript

"water pouring out of the sink and the woman is washing dishes the child is a honest or falling off as he's reaching for the cookie jar was right hand and Friday and the other cookies to a little girl who's reaching up with her left hand to get the cookie and her rice and looks like she's almost trying to make us signal Silence of the mother doesn't hear it from others was here in the water is it pouring on the floor and the window is open you can see a yard outside is Edition cups on this thing and the curtains are pulled back to you can see out the window the cupboard doors open and they're all fully dressed and they have sort of a nondescript expression on her face the woman has her mouth open a little bit which wasn't having other than a little bit "

In [114]:
transcript = "water pouring out of the sink and the woman is washing dishes the child is a honest or falling off as he's reaching for the cookie jar was right hand and Friday and the other cookies to a little girl who's reaching up with her left hand to get the cookie and her rice and looks like she's almost trying to make us signal Silence of the mother doesn't hear it from others was here in the water is it pouring on the floor and the window is open you can see a yard outside is Edition cups on this thing and the curtains are pulled back to you can see out the window the cupboard doors open and they're all fully dressed and they have sort of a nondescript expression on her face the woman has her mouth open a little bit which wasn't having other than a little bit "

In [115]:
transcript = [transcript]

In [116]:
transcript

["water pouring out of the sink and the woman is washing dishes the child is a honest or falling off as he's reaching for the cookie jar was right hand and Friday and the other cookies to a little girl who's reaching up with her left hand to get the cookie and her rice and looks like she's almost trying to make us signal Silence of the mother doesn't hear it from others was here in the water is it pouring on the floor and the window is open you can see a yard outside is Edition cups on this thing and the curtains are pulled back to you can see out the window the cupboard doors open and they're all fully dressed and they have sort of a nondescript expression on her face the woman has her mouth open a little bit which wasn't having other than a little bit "]

In [117]:
bert_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_name,
                                        add_special_tokens=True,
                                        do_lower_case=True,
                                        max_length=256,
                                        pad_to_max_length=True)
def bert_encoder(input_text):
    # txt = input_text.numpy().decode('utf-8')
    txt = input_text
    encoded = tokenizer.encode_plus(txt, add_special_tokens=True, 
                                    max_length=256, 
                                    pad_to_max_length=True, 
                                    return_attention_mask=True, 
                                    return_token_type_ids=True,
                                    truncation=True)
    return encoded['input_ids'], encoded['token_type_ids'], \
        encoded['attention_mask']

bert_train = [bert_encoder(r) for r in transcript]

In [120]:
bert_train = np.array(bert_train)
sc_reviews, sc_segments, sc_masks = np.split(bert_train, 3, axis=1)

In [125]:
model_reload.predict(sc_reviews.squeeze().reshape(1, 256), batch_size=1)

array([[25.492697]], dtype=float32)

In [1]:
import json
import requests

In [127]:
data = json.dumps({
        "instances": sc_reviews.squeeze().reshape(1, 256).tolist()
        })

In [128]:
headers = {"content-type": "application/json"}
response = requests.post('http://' + 'model_server' + ':8501/v1/models/bert:predict', data=data, headers=headers, 
                         timeout=None)

In [129]:
response

<Response [200]>

In [130]:
print(response.text)

{
    "predictions": [[25.4926872]
    ]
}
